In [1]:
import pandas as pd
import numpy as np
import requests
import tempfile
import certifi

# Download PWT data
url = 'https://www.rug.nl/ggdc/docs/pwt90.dta'  # Adjust the URL for the correct version
response = requests.get(url, verify=certifi.where())
response.raise_for_status()

# Save to a temporary file
with tempfile.NamedTemporaryFile(delete=False, suffix='.dta') as tmp:
    tmp.write(response.content)
    tmp_path = tmp.name

# Load the data using pandas
pwt_data = pd.read_stata(tmp_path)

# List of countries from your table
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 
             'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 
             'New Zealand', 'Norway', 'Portugal', 'Spain', 'Sweden', 'Switzerland', 
             'United Kingdom', 'United States']

# Filter data for the years 1990-2019 and specific countries
pwt_filtered = pwt_data[(pwt_data['year'] >= 1990) & (pwt_data['year'] <= 2019) & 
                        (pwt_data['country'].isin(countries))]

# Calculate logarithmic values for real GDP, TFP, and capital stock
pwt_filtered['log_rtfpna'] = np.log(pwt_filtered['rtfpna'])
pwt_filtered['log_rkna'] = np.log(pwt_filtered['rkna'])

# Compute growth rates using the difference in logarithms
growth_rates = pwt_filtered.groupby('country').apply(lambda df: df[['log_rtfpna', 'log_rkna']].diff().dropna().mean()) * 100

# Calculate the calculated Growth Rate as the sum of TFP Growth and Capital Deepening
growth_rates['Calculated Growth Rate'] = growth_rates['log_rtfpna'] + growth_rates['log_rkna']

# Prepare the final table
results = pd.DataFrame({
    'Country': growth_rates.index,
    'Growth Rate': growth_rates['Calculated Growth Rate'],
    'TFP Growth': growth_rates['log_rtfpna'],
    'Capital Deepening': growth_rates['log_rkna']
})
# Calculate TFP share and Capital share
results['TFP Share'] = results['TFP Growth'] / results['Growth Rate']
results['Capital Share'] = results['Capital Deepening'] / results['Growth Rate']

# Print the updated results with the new shares
print(results[['Country', 'Growth Rate', 'TFP Growth', 'Capital Deepening', 'TFP Share', 'Capital Share']])


                       Country  Growth Rate  TFP Growth  Capital Deepening  \
country                                                                      
Australia            Australia     4.125462    0.863510           3.261952   
Austria                Austria     2.802531    0.396777           2.405755   
Belgium                Belgium     2.477305   -0.022236           2.499541   
Canada                  Canada     3.250797    0.185191           3.065606   
Denmark                Denmark     2.271351    0.301330           1.970021   
Finland                Finland     2.537510    0.647673           1.889837   
France                  France     2.540758    0.327812           2.212945   
Germany                Germany     2.695904    0.780926           1.914978   
Greece                  Greece     1.608502   -0.251409           1.859911   
Iceland                Iceland     3.028540    0.724114           2.304427   
Ireland                Ireland     5.949166    1.501448         

/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_76731/2363977375.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt_filtered['log_rtfpna'] = np.log(pwt_filtered['rtfpna'])
/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_76731/2363977375.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pwt_filtered['log_rkna'] = np.log(pwt_filtered['rkna'])
/var/folders/_1/6pq414md0y1_mgwvl5dlnd2h0000gn/T/ipykernel_76731/2363977375.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on 

In [2]:
# Export results to CSV file
results.to_csv('growth_accounting_results.csv', index=False)
